This code was borrowed from AAI 540 labs

https://github.com/mechristenson/aai-540-labs.git

## Create S3 Bucket

In [1]:
#!pip install --upgrade boto3 botocore awscli

In [2]:
import boto3
import sagemaker

session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
setup_s3_bucket_passed = False

In [4]:
print("Default bucket: {}".format(bucket))

Default bucket: sagemaker-us-east-1-590183687297


Verify S3_BUCKET Bucket Creation

In [5]:
from botocore.client import ClientError

response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

{'ResponseMetadata': {'RequestId': '22A44Q29S48394A3', 'HostId': 'DilCGa8B3Vbn0czvhWbKof5c+yoq/EqUvkAnya5cC+rSEJISj6CEjbf5Oy/CGGsmRYaR2EKrhFJCStbALHMtUg==', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'DilCGa8B3Vbn0czvhWbKof5c+yoq/EqUvkAnya5cC+rSEJISj6CEjbf5Oy/CGGsmRYaR2EKrhFJCStbALHMtUg==', 'x-amz-request-id': '22A44Q29S48394A3', 'date': 'Wed, 24 Sep 2025 02:27:27 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'BucketRegion': 'us-east-1', 'AccessPointAlias': False}


In [6]:
%store setup_s3_bucket_passed


Stored 'setup_s3_bucket_passed' (bool)


In [7]:
%store


Stored variables and their in-db values:
ingest_create_athena_db_passed             -> True
s3_private_path_csv                        -> 's3://sagemaker-us-east-1-590183687297/spotify_tra
s3_private_path_parquet                    -> 's3://sagemaker-us-east-1-590183687297/toxicity_pd
setup_s3_bucket_passed                     -> True


## Set up Data lake

In [8]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

#### Set S3 Destination Folder

In [9]:
s3_private_path_parquet = "s3://{}/toxicity_pds/parquet".format(bucket)
print(s3_private_path_parquet)

s3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet


In [10]:
%store s3_private_path_parquet

Stored 's3_private_path_parquet' (str)


In [11]:
%store -r setup_s3_bucket_passed

In [12]:
try:
    setup_s3_bucket_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++")


In [13]:
if not setup_s3_bucket_passed:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

### Copy data from github to S3 bucket

In [14]:
!aws s3 cp --recursive /home/sagemaker-user/aai540_toxicity_classification/civil $s3_private_path_parquet/ --include "*"


upload: civil/validation-00000-of-00001.parquet to s3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet/validation-00000-of-00001.parquet
upload: civil/test-00000-of-00001.parquet to s3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet/test-00000-of-00001.parquet
upload: civil/train-00000-of-00001.parquet to s3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet/train-00000-of-00001.parquet


### List the files

In [15]:
print(s3_private_path_parquet)

s3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet


In [16]:
!aws s3 ls $s3_private_path_parquet/

2025-09-24 02:27:35   34099179 test-00000-of-00001.parquet
2025-09-24 02:27:35   68844404 train-00000-of-00001.parquet
2025-09-24 02:27:35   11697541 validation-00000-of-00001.parquet


In [17]:
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed             -> True
s3_private_path_csv                        -> 's3://sagemaker-us-east-1-590183687297/spotify_tra
s3_private_path_parquet                    -> 's3://sagemaker-us-east-1-590183687297/toxicity_pd
setup_s3_bucket_passed                     -> True


## Create Athena Database Schema

In [17]:
ingest_create_athena_db_passed = False

In [18]:
%store -r s3_private_path_parquet

In [19]:
try:
    s3_private_path_parquet
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS COPY TSV TO S3 NOTEBOOK ******************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")

In [20]:
print(s3_private_path_parquet)

s3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet


### Import PyAthena

In [21]:
from pyathena import connect

### Create Athena Database

In [22]:
database_name = "aai540_toxicity_aws"

In [23]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [24]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [25]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS aai540_toxicity_aws


In [26]:
import pandas as pd

pd.read_sql(statement, conn)

/tmp/ipykernel_264/3803073958.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


### Verify The Database Has Been Created Succesfully

In [27]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_264/3999478089.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,aai540_spotify_aws
1,aai540_toxicity_aws
2,default
3,sagemaker_featurestore


In [28]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

In [29]:
%store ingest_create_athena_db_passed

Stored 'ingest_create_athena_db_passed' (bool)


In [30]:
%store

Stored variables and their in-db values:
ingest_create_athena_db_passed             -> True
s3_private_path_csv                        -> 's3://sagemaker-us-east-1-590183687297/spotify_tra
s3_private_path_parquet                    -> 's3://sagemaker-us-east-1-590183687297/toxicity_pd
setup_s3_bucket_passed                     -> True


### Create Tables

In [31]:
ingest_create_athena_table_pqt_passed = False

In [32]:
table_name = 'toxicity_pqt'
train = '%/train-00000-of-00001.parquet'
validation = '%/validation-00000-of-00001.parquet'
test = '%/test-00000-of-00001.parquet'

In [33]:
s3_private_path_parquet

's3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet'

In [34]:
drop_stmt = """DROP TABLE IF EXISTS {}.{}""".format(database_name, table_name)
pd.read_sql(drop_stmt, conn)

/tmp/ipykernel_264/2386943986.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(drop_stmt, conn)


""


In [35]:
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{} (
  uid               BIGINT,
  id                BIGINT,
  comment_text      STRING,
  toxicity          BIGINT,
  has_active_attrs  BOOLEAN,
  active_attrs      ARRAY<string>,
  male              BIGINT,
  female            BIGINT,
  lgbtq             BIGINT,
  christian         BIGINT,
  muslim            BIGINT,
  other_religions   BIGINT,
  black             BIGINT,
  white             BIGINT,
  identity_any      BIGINT,
  severe_toxicity   BIGINT,
  obscene           BIGINT,
  threat            BIGINT,
  insult            BIGINT,
  identity_attack   BIGINT,
  sexual_explicit   BIGINT
)
STORED AS PARQUET
LOCATION '{}/'""".format(database_name, table_name, s3_private_path_parquet)

print(statement)
pd.read_sql(statement, conn)

CREATE EXTERNAL TABLE IF NOT EXISTS aai540_toxicity_aws.toxicity_pqt (
  uid               BIGINT,
  id                BIGINT,
  comment_text      STRING,
  toxicity          BIGINT,
  has_active_attrs  BOOLEAN,
  active_attrs      ARRAY<string>,
  male              BIGINT,
  female            BIGINT,
  lgbtq             BIGINT,
  christian         BIGINT,
  muslim            BIGINT,
  other_religions   BIGINT,
  black             BIGINT,
  white             BIGINT,
  identity_any      BIGINT,
  severe_toxicity   BIGINT,
  obscene           BIGINT,
  threat            BIGINT,
  insult            BIGINT,
  identity_attack   BIGINT,
  sexual_explicit   BIGINT
)
STORED AS PARQUET
LOCATION 's3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet/'


/tmp/ipykernel_264/117541428.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [36]:
train_view = 'toxicity_train'
statement_view = """CREATE OR REPLACE VIEW {}.{} AS SELECT * FROM {}.{} WHERE "$path" LIKE '{}'""".format(
    database_name,
    train_view,
    database_name,
    table_name,
    train
)
print(statement_view)
pd.read_sql(statement_view, conn)

CREATE OR REPLACE VIEW aai540_toxicity_aws.toxicity_train AS SELECT * FROM aai540_toxicity_aws.toxicity_pqt WHERE "$path" LIKE '%/train-00000-of-00001.parquet'


/tmp/ipykernel_264/2100174071.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement_view, conn)


""


In [37]:
validation_view = 'toxicity_val'
statement_view = """CREATE OR REPLACE VIEW {}.{} AS SELECT * FROM {}.{} WHERE "$path" LIKE '{}'""".format(
    database_name,
    validation_view,
    database_name,
    table_name,
    validation
)
print(statement_view)
pd.read_sql(statement_view, conn)

CREATE OR REPLACE VIEW aai540_toxicity_aws.toxicity_val AS SELECT * FROM aai540_toxicity_aws.toxicity_pqt WHERE "$path" LIKE '%/validation-00000-of-00001.parquet'


/tmp/ipykernel_264/3418994591.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement_view, conn)


""


In [38]:
test_view = 'toxicity_test'
statement_view = """CREATE OR REPLACE VIEW {}.{} AS SELECT * FROM {}.{} WHERE "$path" LIKE '{}'""".format(
    database_name,
    test_view,
    database_name,
    table_name,
    test
)
print(statement_view)
pd.read_sql(statement_view, conn)


CREATE OR REPLACE VIEW aai540_toxicity_aws.toxicity_test AS SELECT * FROM aai540_toxicity_aws.toxicity_pqt WHERE "$path" LIKE '%/test-00000-of-00001.parquet'


/tmp/ipykernel_264/1627434926.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement_view, conn)


""


### Verify The Table Has Been Created Succesfully

In [39]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_264/2201015668.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,toxicity_pqt
1,toxicity_test
2,toxicity_train
3,toxicity_val


In [40]:
if table_name in df_show.values:
    ingest_create_athena_table_pqt_passed = True

print(ingest_create_athena_table_pqt_passed)

True


### Run A Sample SQL Query

In [41]:
statement = """SELECT * FROM {}.{}""".format(
    database_name, table_name
)

print(statement)

df = pd.read_sql(statement, conn)
df.head(5)

SELECT * FROM aai540_toxicity_aws.toxicity_pqt


/tmp/ipykernel_264/2419674085.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,uid,id,comment_text,toxicity,has_active_attrs,active_attrs,male,female,lgbtq,christian,...,other_religions,black,white,identity_any,severe_toxicity,obscene,threat,insult,identity_attack,sexual_explicit
0,18,5711489,𝑯𝒆 𝒕𝒐𝒍𝒅 𝒑𝒐𝒍𝒊𝒄𝒆 𝒕𝒉𝒂𝒕 𝒉𝒆 𝒃𝒖𝒚𝒔 𝒄𝒐𝒄𝒂𝒊𝒏𝒆 “𝒊𝒏 𝒍𝒂𝒓𝒈𝒆 ...,0,False,[],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,21,5295059,为加拿大和中国友谊祝贺。与其想象中国人对加拿大人的看法不如直接去问，不要用自己的想法去猜。本...,0,True,[christian],0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
2,26,5753025,♫ Mister we could use a man like Neville Chamb...,0,True,[male],1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,47,6100226,…so much for your opinion.,0,False,[],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,54,974066,…as most everyone knows both parties completel...,0,False,[],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
statement = "SHOW VIEWS in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_264/3130455028.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,views
0,toxicity_test
1,toxicity_train
2,toxicity_val


In [45]:
df['toxicity'].value_counts()

toxicity
0    394821
1     50472
Name: count, dtype: int64

## Set up Sagemaker Feature store

In [18]:
from sagemaker.session import Session

region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)

featurestore_runtime = boto_session.client(
    service_name="sagemaker-featurestore-runtime", region_name=region
)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime,
)

### S3 Bucket set up for offline store

In [19]:
# You can modify the following to use a bucket of your choosing
default_s3_bucket_name = feature_store_session.default_bucket()
print(default_s3_bucket_name)

sagemaker-us-east-1-590183687297


In [20]:
# set up IAM role
from sagemaker import get_execution_role

# You can modify the following to use a role of your choosing. See the documentation for how to create this.
role = get_execution_role()
print(role)

arn:aws:iam::590183687297:role/LabRole


In [21]:
s3_private_path_parquet

's3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet'

In [22]:
s3_private_path_parquet.replace("s3://", "").split("/", 1)

['sagemaker-us-east-1-590183687297', 'toxicity_pds/parquet']

In [23]:
!aws s3 ls $s3_private_path_parquet/

2025-09-24 02:27:35   34099179 test-00000-of-00001.parquet
2025-09-24 02:27:35   68844404 train-00000-of-00001.parquet
2025-09-24 02:27:35   11697541 validation-00000-of-00001.parquet


In [24]:
import boto3
import pandas as pd
import io

# Your region & private bucket
s3_client = boto3.client("s3", region_name=region)

# Split the s3 path into bucket and prefix
# Example: if $s3_private_path_csv="my-private-bucket/housing-data"
bucket_name, prefix = s3_private_path_parquet.replace("s3://", "").split("/", 1)

# Define the file keys (relative to prefix)
train_file_key = f"{prefix}/train-00000-of-00001.parquet"
validation_file_key = f"{prefix}/validation-00000-of-00001.parquet"
test_file_key = f"{prefix}/test-00000-of-00001.parquet"


# Load the files
train_data_object = s3_client.get_object(Bucket=bucket_name, Key=train_file_key)
validation_data_object = s3_client.get_object(Bucket=bucket_name, Key=validation_file_key)
test_data_object = s3_client.get_object(Bucket=bucket_name, Key=test_file_key)

# Convert to pandas DataFrames
train_data = pd.read_parquet(io.BytesIO(train_data_object["Body"].read()))
validation_data = pd.read_parquet(io.BytesIO(validation_data_object["Body"].read()))
test_data = pd.read_parquet(io.BytesIO(test_data_object["Body"].read()))


print("Train Data Shape:", train_data.shape)
print("Validation Data Shape:", validation_data.shape)
print("Test Data Shape:", test_data.shape)



Train Data Shape: (267516, 21)
Validation Data Shape: (45047, 21)
Test Data Shape: (132730, 21)


In [25]:
civil = pd.concat([train_data, validation_data, test_data], axis=0)
civil.shape

(445293, 21)

In [26]:
# Sample 60,000 records where toxicity == 0
civil_tox0 = civil[civil['toxicity'] == 0].sample(n=60000, random_state=42)
# Get all records where toxicity == 1
civil_tox1 = civil[civil['toxicity'] == 1]
# Concatenate to create a balanced dataframe
civil_balanced = pd.concat([civil_tox0, civil_tox1], ignore_index=True)
# Shuffle the resulting dataframe
civil_df = civil_balanced.sample(frac=1, random_state=42).reset_index(drop=True)
civil_df['toxicity'].value_counts()

toxicity
0    60000
1    50472
Name: count, dtype: int64

### Feature Engineering

In [27]:
import re

TAG_RE = re.compile(r'<[^>]+>')

def re_tags(text_list): #define remove tag funtion
    return [TAG_RE.sub('', str(word)).lower() for word in text_list]

In [44]:
# Remove Accented Characters

import unidecode
def re_accented_char(text_list):
    return [unidecode.unidecode(word.encode().decode('utf-8')) for word in text_list]

In [41]:
# Extended contractions
def ex_contractions(text_list):
    result=[]
    for word in text_list:
        # replace contracting withoutsignal
        word = word.replace("wont","won't")
        word = word.replace("cant","can't")
        word = word.replace("its","it's")
        word = word.replace("youre","you're")
        word = word.replace("hes","he's")
        word = word.replace("shes","she's")
        word = word.replace("its","it's")
        word = word.replace("weare","we're")
        word = word.replace("theyre","they're")

        # specific
        word = re.sub(r"won\'t", "will not", str(word))
        word = re.sub(r"can\'t", "can not", str(word))

        # general
        word = re.sub(r"n\'t", " not", str(word))
        word = re.sub(r"\'re", " are", str(word))
        word = re.sub(r"\'s", " is", str(word))
        word = re.sub(r"\'d", " would", str(word))
        word = re.sub(r"\'ll", " will", str(word))
        word = re.sub(r"\'t", " not", str(word))
        word = re.sub(r"\'ve", " have", str(word))
        word = re.sub(r"\'m", " am", str(word))
        result.append(word)
    return result

In [29]:
# Removing Special Characters
def re_special_chars(text_list):
    return [re.sub("[^a-zA-Z0-9]"," ",word) for word in text_list]

#### Lemmatization

In [30]:
from nltk.stem import WordNetLemmatizer

def lemmatize_text(text_list):
    wnl = WordNetLemmatizer()
    lemmatizer_sentence = []  
    tokenizer=nltk.tokenize.WhitespaceTokenizer()
    for word in tokenizer.tokenize(text_list):
        lemmatizer_sentence.append(wnl.lemmatize(word,'v'))
        lemmatizer_sentence.append(" ")
    
    return("".join(lemmatizer_sentence))

#### Removing Stop Words

In [31]:
import nltk
from nltk.corpus import stopwords

def stopwords_text(text_list):
    stop = stopwords.words('english')
    sentence_without = []
    tokenizer=nltk.tokenize.WhitespaceTokenizer()
    for word in tokenizer.tokenize(text_list):
        if word not in stop:
            sentence_without.append(word)
            sentence_without.append(" ")
            
    return("".join(sentence_without))

In [32]:
def re_whitespaces(text_list): 
    result=[]
    for word in text_list:
        word=(re.sub(r'\d','',str(word))) #remove numbers 
        word = (re.sub(r'\s+',' ', str(word))) #remove duplicates white spacces
        result.append(word)
    return result

In [33]:
# Removing blank comments
civil_df = civil_df[civil_df['comment_text']!='']

### Pipeline

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [35]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sagemaker-
[nltk_data]     user/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
# Class for regular expressions application
class ApplyRegex(BaseEstimator, TransformerMixin):
    
    def __init__(self, regex_transformers):
        self.regex_transformers = regex_transformers
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        # Applying all regex functions in the regex_transformers dictionary
        for regex_name, regex_function in self.regex_transformers.items():
            X = regex_function(X)
            
        return X

In [37]:
class StopWordsRemoval(BaseEstimator, TransformerMixin):
    
    def __init__(self, text_stopwords):
        self.text_stopwords = text_stopwords
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return [self.text_stopwords(comment) for comment in X]

In [38]:
class LemmatizeProcess(BaseEstimator, TransformerMixin):
    
    def __init__(self, Lemmatize):
        self.Lemmatizer = Lemmatize
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return [self.Lemmatizer(comment) for comment in X]

In [45]:
# Defining regex transformers to be applied
regex_transformers = {
    'remove_tags': re_tags,
    'remove_accents': re_accented_char,
    'decontracted': ex_contractions,
    're_sc': re_special_chars,
    'whitespaces': re_whitespaces
}

# Building a text prep pipeline
text_prep_pipeline = Pipeline([
    ('regex', ApplyRegex(regex_transformers)),
    ('stopwords', StopWordsRemoval(stopwords_text)),
    ('lemmatize', LemmatizeProcess(lemmatize_text)),
])

In [46]:
civil_df

,uid,id,comment_text,toxicity,has_active_attrs,active_attrs,male,female,LGBTQ,christian,...,other_religions,black,white,identity_any,severe_toxicity,obscene,threat,insult,identity_attack,sexual_explicit
0,54642,544615,"While praising a Jesuit pope and the Jesuit ""t...",0,True,[male],1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,138661,5937757,That simply proves that lib trolls are on this...,1,False,[],0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,116897,5685492,The paranoia of the belief Americans are all r...,1,True,"[female, white]",0,1,0,0,...,0,0,1,1,0,0,0,1,0,0
3,363315,7117052,David Brooks the original Israel firster and h...,1,False,[],0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,273358,5309837,"Iditarod dogs are loved, honored and even give...",1,False,[],0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110467,232287,5378642,Like father like son. Trudeau oh so eager to ...,1,False,[],0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
110468,437996,5815651,"""I get it.\nTrump is a bad guy.""\n\nNo, you cl...",1,False,[],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
110469,222628,5461945,"Mike objects to gay sex because it's ""icky"". H...",1,True,[LGBTQ],0,0,1,0,...,0,0,0,1,0,0,0,1,1,0
110470,150178,5255638,Sounds so joyous!\nGreat Mother's Day story.,0,False,[],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
civil_df[civil_df.columns[2:3]]

,comment_text
0,"While praising a Jesuit pope and the Jesuit ""t..."
1,That simply proves that lib trolls are on this...
2,The paranoia of the belief Americans are all r...
3,David Brooks the original Israel firster and h...
4,"Iditarod dogs are loved, honored and even give..."
...,...
110467,Like father like son. Trudeau oh so eager to ...
110468,"""I get it.\nTrump is a bad guy.""\n\nNo, you cl..."
110469,"Mike objects to gay sex because it's ""icky"". H..."
110470,Sounds so joyous!\nGreat Mother's Day story.


In [48]:
civil_df['comment_text'] = text_prep_pipeline.fit_transform(civil_df[civil_df.columns[2:3]].values)

In [49]:
civil_df['comment_text']

0         praise jesuit pope jesuit magis remind pick je...
1          nothat simply prove lib troll site instead work 
2         paranoia belief americans racist never end dum...
3         wouldavid brook original israel firster hater ...
4         iditarod dog love honor even give life race ni...
                                ...                        
110467    like father like son trudeau oh eager mollify ...
110468                    get ntrump bad guy n nno clearly 
110469    amike object gay sex icky think justify fear l...
110470                sound joyous ngreat mother day story 
110471                                  catholic press get 
Name: comment_text, Length: 110472, dtype: object

#### Define FeatureGroups¶

In [94]:
text_feature_group_name = "text-feature-group"
identity_feature_group_name = "identity-feature-group"
label_feature_group_name = "label-feature-group"

In [104]:
from sagemaker.feature_store.feature_group import FeatureGroup

text_feature_group = FeatureGroup(
    name=text_feature_group_name, sagemaker_session=feature_store_session
)
identity_feature_group = FeatureGroup(
    name=identity_feature_group_name, sagemaker_session=feature_store_session
)
label_feature_group = FeatureGroup(
    name=label_feature_group_name, sagemaker_session=feature_store_session
)

In [105]:
text_df = civil_df[['id','comment_text','toxicity']]
identity_df = civil_df[['id','male','female','LGBTQ','christian','muslim','other_religions','black','white']]
label_df = civil_df[['id','toxicity','severe_toxicity','obscene','threat','insult','identity_attack','sexual_explicit']]

In [106]:
import time

current_time_sec = int(round(time.time()))

def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == "object":
            data_frame[label] = data_frame[label].astype("str").astype("string")
    return data_frame


def appendEventTime(df):
    # append EventTime feature
    df[event_time_feature_name] = pd.Series(
        [current_time_sec] * len(df), dtype="float64"
    )
    return df

In [107]:
# cast object dtype to string. The SageMaker FeatureStore Python SDK will then map the string dtype to String feature type.
text_df = cast_object_to_string(text_df)
identity_df = cast_object_to_string(identity_df)
label_df = cast_object_to_string(label_df)

/tmp/ipykernel_157/2537523605.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame[label] = data_frame[label].astype("str").astype("string")


In [108]:
record_identifier_feature_name = "id"
event_time_feature_name = "EventTime"

text_df = appendEventTime(text_df)
identity_df = appendEventTime(identity_df)
label_df = appendEventTime(label_df)

/tmp/ipykernel_157/2537523605.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[event_time_feature_name] = pd.Series(
/tmp/ipykernel_157/2537523605.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[event_time_feature_name] = pd.Series(
/tmp/ipykernel_157/2537523605.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [109]:
# load feature definitions to the feature group. SageMaker FeatureStore Python SDK will auto-detect the data schema based on input data.
text_feature_group.load_feature_definitions(data_frame=text_df)
identity_feature_group.load_feature_definitions(data_frame=identity_df)
label_feature_group.load_feature_definitions(data_frame=label_df)

[FeatureDefinition(feature_name='id', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='toxicity', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='severe_toxicity', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='obscene', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='threat', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='insult', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='identity_attack', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_name='sexual_explicit', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None),
 FeatureDefinition(feature_nam

In [118]:
def featureGroupExists(feature_group_name):
    print(feature_group_name)
    # SageMaker client (not session)
    sm_client = boto3.client("sagemaker", region_name=region)
    featureGroupExists = False
    try:
        response = sm_client.describe_feature_group(FeatureGroupName=feature_group_name)
        featureGroupExists = True
        print(f"✅ Feature group exists. Status: {response['FeatureGroupStatus']}")
    except sm_client.exceptions.ResourceNotFound:
        featureGroupExists=False
        print(f"❌ {feature_group_name} does not exist.")
    return featureGroupExists

text_feature_group_exists = featureGroupExists(text_feature_group_name)
identity_feature_group_exists = featureGroupExists(identity_feature_group_name)
label_feature_group_exists = featureGroupExists(label_feature_group_name)

text-feature-group
✅ Feature group exists. Status: Created
identity-feature-group
❌ identity-feature-group does not exist.
label-feature-group
❌ label-feature-group does not exist.


In [122]:
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")

def create_feature_group(feature_group, fg_Exists):
    fg_Exists = featureGroupExists(feature_group.name)
    if not fg_Exists:
        feature_group.create(
            s3_uri=f"s3://{default_s3_bucket_name}/{prefix}",
            record_identifier_name=record_identifier_feature_name,
            event_time_feature_name=event_time_feature_name,
            role_arn=role,
            enable_online_store=True,
        )
        wait_for_feature_group_creation_complete(feature_group)
    else:
        print(f"{feature_group.name} already exists")
    

create_feature_group(text_feature_group,text_feature_group_exists)
create_feature_group(identity_feature_group, identity_feature_group_exists)
create_feature_group(label_feature_group,label_feature_group_exists)



text-feature-group
✅ Feature group exists. Status: Created
text-feature-group already exists
identity-feature-group
❌ identity-feature-group does not exist.
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup identity-feature-group successfully created.
label-feature-group
❌ label-feature-group does not exist.
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup label-feature-group successfully created.


In [123]:
text_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:590183687297:feature-group/text-feature-group',
 'FeatureGroupName': 'text-feature-group',
 'RecordIdentifierFeatureName': 'id',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'id', 'FeatureType': 'Integral'},
  {'FeatureName': 'comment_text', 'FeatureType': 'String'},
  {'FeatureName': 'toxicity', 'FeatureType': 'Integral'},
  {'FeatureName': 'EventTime', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2025, 9, 24, 4, 17, 43, 61000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': 's3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet',
   'ResolvedOutputS3Uri': 's3://sagemaker-us-east-1-590183687297/toxicity_pds/parquet/590183687297/sagemaker/us-east-1/offline-store/text-feature-group-1758687463/data'},
  'DisableGlueTableCreation': False,
  'DataCatalogConfig': {'TableName': 'text_feature_group_1758687463',
 

### Release Resources

In [1]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [1]:

%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

<IPython.core.display.Javascript object>